In [1]:
import numpy as np
import math

# 冷蔵庫　JIS C 9801：1999準拠APF

## 状態値と消費電力

### 状態値
JIS C 9801：1999に準拠したAPFを表示している冷蔵庫は、以下の状態値を持つ。

| 状態値 | 説明 |   
|:-------:|:----:|    
|0        |停止 |  
|1        |除霜 |
|2        |冷蔵 |

### 状態値0の消費電力

状態値0は停止を意味し、その消費電力$P_{elc,state,0}$は0 W である。

$$
P_{elc,state,0}　= 0
$$

In [2]:
def get_P_elc_stat_0():
    return 0

In [3]:
get_P_elc_stat_0()

0

### 状態値1の消費電力

状態値1は除霜を意味し、その消費電力$P_{elc,state,1}$は除霜用伝熱器の消費電力$P_{elc,htr}$に補正係数$C_{crct,htr}$を掛けた値である。
 
$$
P_{elc,state,1} = P_{htr,rtd} \times C_{crct,htr}
$$
  
$P_{htr,rtd}$ :除霜用電熱器(<u>h</u>ea<u>t</u>e<u>r</u>)の定格(<u>r</u>a<u>t</u>e<u>d</u>)消費電力 [W]  
$C_{crct,htr}$ :除霜用電熱器の定格消費電力に対する補正係数(<u>c</u>o<u>rr</u>e<u>ct</u>ion <u>c</u>oefficient) (=0.9) [-]  

In [4]:
def get_P_elc_stat_1(P_htr_rtd):
    CORRECTION_COEFFICIENT_OF_HEATER = 0.9
    return P_htr_rtd * CORRECTION_COEFFICIENT_OF_HEATER

In [5]:
get_P_elc_stat_1(60)

54.0

### 状態値2の消費電力

状態値2は冷蔵を意味し、その消費電力$P_{elc,rfrg}$は$d$日の冷蔵庫の推定消費電力量から除霜運転の消費電力量を引いた値を冷蔵運転時間で除した値である。

1日当たりの除霜運転は2時間とする。

$d$日の1日当たりの冷蔵庫の推定消費電力量$E_{elc,est,d}$は冷蔵庫の年間消費電力量と冷蔵庫周辺空気の日平均温度を説明変数とした推定値とし、式xで求まる。 

$d$日の冷蔵庫の周辺空気の日平均温度は日平均外気温度と住宅の熱損失係数を説明変数とした推定値とする。  

$$
P_{elc,state,2} = \frac{E_{elc,est,d} - P_{elc,dfrs} \times hr_{dfrs}}{24 - hr_{dfrs}}  
$$  

$$
P_{elc,dfrs} = P_{elc,state,1}
$$
  
$$
E_{elc,est,d} = [(3.283 \times 10^{-3} -2.0 \times 10^{-6} \times APF)
\times (\theta_{amb,est,d}^{2} - 30 \times \theta_{amb,est,d}) + 1.85 \times 10^{-3} \times APF + 1.329 ] \times 10^{3}
$$  
$d$日の冷蔵庫の推定消費電力はJIS基準での1日別消費電力に直すべき。

$$
\theta_{amb,est,d} =
\begin{cases}
(0.048 \times Q + 0.3666) \times \theta_{ext,d} + (-1.2435 \times Q + 18.82) & ,居室間歇暖房時 \\
(0.0604 \times Q + 0.3668) \times \theta_{ext,d} + (-1.6265 \times Q + 19.99) & ,全館連続暖房時 \\
\end{cases}
$$
$\theta_{amb,est,d}$を求める過程で暖房方式が指定されるのは、現時点では仕方ないが、煩雑に思われる。  

$E_{elc,est,d}$ :$d$日の冷蔵庫の推定(<u>est</u>imated)消費電力量 [Wh/d]  
$hr_{dfrs}$ :1日当たりの除霜運転の時間 (= 2) [h/d]  
$E_{elc,anl}$ :冷蔵庫の年間(<u>ann</u>ua<u>l</u>)消費電力量 [kWh/y]  
$\theta_{amb,est,d}$ :$d$日の冷蔵庫周辺空気(<u>amb</u>ient air)の日平均温度 [℃]  
$\theta_{ext,d}$ :$d$日の日平均外気(<u>ext</u>ernal air)温度 [℃]  
$Q$ :住宅の熱損失係数 [W/(m<sup>2</sup>・K)]  


In [6]:
#本来は外部からインポートする
Q = 2.5

In [7]:
#本来は外部からインポートする
day_number = np.arange(365)
daily_average_external_air_temperature = np.random.normal(20,20,365)
DAEAT = dict(zip(day_number,daily_average_external_air_temperature))
DAEAT
#DailyAverageExternalAirTemperature


{0: 34.83570326231176,
 1: 94.470530448603952,
 2: 52.191295098718967,
 3: 41.094570509440231,
 4: 36.662359306324625,
 5: 23.005171920005527,
 6: -28.704714180818925,
 7: 9.6562849636014789,
 8: 10.29123615736442,
 9: 15.330924743861065,
 10: -3.1342254132327554,
 11: 6.5176561803586992,
 12: 32.951395638859005,
 13: 32.199549554231595,
 14: 23.829471429371139,
 15: 0.28785820385058969,
 16: -16.315564836467942,
 17: 2.6173719041708061,
 18: 35.761945617205896,
 19: 31.292107379091057,
 20: 51.386832905996798,
 21: -11.603889367742045,
 22: 4.8790631036738823,
 23: 26.114487048070004,
 24: 24.244936032922318,
 25: -9.0896948994703628,
 26: 44.396378587167547,
 27: 25.735224032427325,
 28: 14.554153218930185,
 29: 41.417877969769151,
 30: 19.689325308567284,
 31: 34.111071177419035,
 32: 7.1857579400460398,
 33: 60.614148701964645,
 34: 36.406698176715679,
 35: 21.369889043588273,
 36: 38.840322304888502,
 37: 26.450080366860789,
 38: 21.954544636797959,
 39: 39.098155114170254,
 40: 4

In [8]:
def get_P_elc_stat_2(APF, P_elc_dfrs_rtd, day_number, central_heating_exists):
    HOURS_OF_DEFROST = 2
    theta_ext_d = DAEAT[day_number]
    if central_heating_exists == False:
        theta_amb_est_d = (0.048 * Q + 0.3666) * theta_ext_d + (-1.2435 * Q + 18.82)
    else:
        theta_amb_est_d = (0.0604 * Q + 0.3668) * theta_ext_d + (-1.6265 * Q + 19.99)
    E_elc_est_d_kWh = (3.283 * 10**(-3) - 2.0 * 10**(-6) * APF) * (theta_amb_est_d**2 - 30 * theta_amb_est_d) \
                      + 1.85 * 10**(-3) * APF \
                      + 1.329
    E_elc_est_d = E_elc_est_d_kWh * 10**(3)

    '''
    print('Q is {}'.format(Q))
    print('theta_ext_d is {}'.format(theta_ext_d))
    print('theta_amb_est_d is {}'.format(theta_amb_est_d))
    print('E_elc_est_d_kWh is {}'.format(E_elc_est_d_kWh))
    '''
    
    return (E_elc_est_d - get_P_elc_stat_1(P_elc_dfrs_rtd) * HOURS_OF_DEFROST) / (24 - HOURS_OF_DEFROST)

In [9]:
get_P_elc_stat_2(6, 50, 3, False)

87.126430621460869

## サンプル

1つの計算ステップのサンプル。実際の運用時には年間結果を配列として返す形になる。

In [10]:
def get_P_elc(state_number, target_date_string):
    
    #/set parameters
    pAPF = 6 #APF
    pPdfrstrtd = 50 #除霜器の定格消費電力
    pCntrlHtExst = False #全館空調が存在するか
    #set parameters/
    
    target_date = np.datetime64(target_date_string)
    dayNum = math.floor((target_date - np.datetime64('2001-01-01T00:00:00')).item().total_seconds() / (3600 * 24))

    if state_number == 0:
        return get_P_elc_stat_0()
    elif state_number == 1:
        return get_P_elc_stat_1(pPdfrstrtd)
    elif state_number == 2:
        return get_P_elc_stat_2(pAPF, pPdfrstrtd, dayNum, pCntrlHtExst)
    else:
        raise ValueError(state_number)
    return 0

In [11]:
get_P_elc(1, '2001-01-05T00:00:00')

45.0